In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

In [5]:
df = pd.read_csv("datasets/libras_minds_dataset_openpose.csv")

In [6]:
df = df.drop(["Unnamed: 0"], axis=1)

In [8]:
person_videos = 15
persons = 5
frames = 30
person = []
for i in range(persons):
    person.extend([i] * frames * person_videos)
person = person * len(df["category"].unique())

In [9]:
df["person"] = person

In [10]:
df

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z,missing_hand,missing_face,person
0,hello,hello_0.avi,0,0.656550,0.932395,0.025427,0.654852,0.936922,0.018061,0.629383,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0
1,hello,hello_0.avi,1,0.654789,0.870996,0.124683,0.676395,0.889949,0.042268,0.640005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0
2,hello,hello_0.avi,2,0.701695,0.784901,0.017858,0.668885,0.802914,0.018611,0.697835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0
3,hello,hello_0.avi,3,0.703596,0.738675,0.033277,0.704249,0.617640,0.028435,0.703596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0
4,hello,hello_0.avi,4,0.722398,0.629388,0.055209,0.721750,0.602609,0.019473,0.653724,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22495,you,you_9.avi,25,0.687613,0.987464,0.052100,0.686957,1.024196,0.028007,0.664655,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,4
22496,you,you_9.avi,26,0.690882,0.988480,0.037995,0.687608,1.024281,0.023716,0.662066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,4
22497,you,you_9.avi,27,0.694779,0.985757,0.023028,0.686354,1.022913,0.024061,0.666912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,4
22498,you,you_9.avi,28,0.688939,0.989507,0.041573,0.685065,1.023943,0.025412,0.661176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,4


In [11]:
df["category"].unique()

array(['hello', 'howAre', 'love', 'mask', 'no', 'please', 'sorry',
       'thanks', 'wear', 'you'], dtype=object)

In [13]:
landmarks_name = np.array([i for i in list(df.columns) if i not in ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2", "category", "category_index", "video_name", "frame", "missing_hand", "missing_face", "person"]])
landmarks_name = landmarks_name.reshape((int(landmarks_name.shape[0]/3), 3))

In [14]:
# GPT Code
def rotate_landmarks(landmarks, rotation_angle):
    # Assuming landmarks are in (x, y, z) format
    
    # Calculate the centroid (center) of the landmarks
#     centroid = np.mean(landmarks, axis=0)
    centroid = 0.5

    # Translate landmarks to the origin (center)
    translated_landmarks = landmarks - centroid
    # Create a rotation matrix for the given angle (in radians)
    rotation_matrix = np.array([[np.cos(rotation_angle), -np.sin(rotation_angle), 0],
                                 [np.sin(rotation_angle), np.cos(rotation_angle), 0],
                                 [0, 0, 1]])
    
    # Apply the rotation to each landmark
#     print(rotation_matrix)
    rotated_landmarks = np.dot(translated_landmarks, rotation_matrix)
    
    return rotated_landmarks + centroid

In [15]:
def zoom_landmarks(landmarks, zoom_factor):
    # Scale the landmarks by the zoom factor
    zoomed_landmarks = landmarks * zoom_factor
    
    return zoomed_landmarks

In [16]:
def translate_landmarks(landmarks, translation_vector):
    # Translate the landmarks by the given vector
    translated_landmarks = landmarks + translation_vector
    
    return translated_landmarks

In [17]:
def apply_transformation(frame, rotation, zoom, translate_x, translate_y):
    for lx, ly, lz in landmarks_name:
        result = rotate_landmarks(frame[[lx, ly, lz]], math.radians(rotation))
        frame[[lx, ly, lz]] = result
        result = zoom_landmarks(frame[[lx, ly, lz]], zoom)
        frame[[lx, ly, lz]] = result
        result = translate_landmarks(frame[[lx, ly, lz]], [translate_x, translate_x, 0])
        frame[[lx, ly, lz]] = result
    return frame

In [18]:
np.random.normal(0, 0.5, 10)

array([-0.18044712, -0.62766709, -0.24312208, -0.31986401, -0.5733274 ,
        0.3433194 , -0.79124067,  0.00454611,  0.88797201, -0.37102728])

In [19]:
sample_size = 0.4
samples_creation = 4
rotation_sigma = 4
zoom_sigma = 0.05
translate_x_sigma = 0.01
translate_y_sigma = 0.0

In [20]:
df_augmented = pd.DataFrame(columns=df.columns)
for category in df["category"].unique():
    print("Processing category", category)
    df_category = df[df["category"] == category]
    videos = df_category["video_name"].unique()
    videos = np.random.choice(videos, int(len(videos) * sample_size))
    for i in range(samples_creation):
        for video in videos:
            df_video = df[df["video_name"] == video].copy()
            rotation = np.random.normal(0, rotation_sigma)
            zoom = np.random.normal(0, zoom_sigma) + 1
            translate_x = np.random.normal(0, translate_x_sigma)
            translate_y = np.random.normal(0, translate_y_sigma)
            df_video = apply_transformation(df_video, rotation, zoom, translate_x, translate_y)
#             for index, frame in df_video.iterrows():
#                 frame = apply_transformation(frame, rotation, zoom, translate_x, translate_y)
#                 df_video[index] = frame
            df_video["video_name"] = f"augmented{i}-" + df_video["video_name"]
            df_augmented = pd.concat([df_augmented, df_video])

Processing category hello
Processing category howAre
Processing category love
Processing category mask
Processing category no
Processing category please
Processing category sorry
Processing category thanks
Processing category wear
Processing category you


In [21]:
df_augmented

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z,missing_hand,missing_face,person
0,hello,augmented0-hello_0.avi,0,0.680723,0.950745,0.025774,0.679075,0.955362,0.018307,0.654449,...,0.0,0.000167,0.016388,0.0,0.000167,0.016388,0.0,False,False,0
1,hello,augmented0-hello_0.avi,1,0.677941,0.888545,0.126386,0.700147,0.907404,0.042845,0.663487,...,0.0,0.000167,0.016388,0.0,0.000167,0.016388,0.0,False,False,0
2,hello,augmented0-hello_0.avi,2,0.724086,0.800524,0.018102,0.691124,0.819313,0.018865,0.719533,...,0.0,0.000167,0.016388,0.0,0.000167,0.016388,0.0,False,False,0
3,hello,augmented0-hello_0.avi,3,0.725262,0.753642,0.033731,0.723961,0.630959,0.028823,0.723595,...,0.0,0.000167,0.016388,0.0,0.000167,0.016388,0.0,False,False,0
4,hello,augmented0-hello_0.avi,4,0.742546,0.642572,0.055963,0.741455,0.615441,0.019739,0.671948,...,0.0,0.000167,0.016388,0.0,0.000167,0.016388,0.0,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20755,you,augmented3-you_23.avi,25,0.092772,-0.067860,0.000000,0.092772,-0.067860,0.000000,0.092772,...,0.0,0.092772,-0.067860,0.0,0.092772,-0.067860,0.0,False,False,1
20756,you,augmented3-you_23.avi,26,0.092772,-0.067860,0.000000,0.092772,-0.067860,0.000000,0.092772,...,0.0,0.092772,-0.067860,0.0,0.092772,-0.067860,0.0,False,False,1
20757,you,augmented3-you_23.avi,27,0.092772,-0.067860,0.000000,0.092772,-0.067860,0.000000,0.092772,...,0.0,0.092772,-0.067860,0.0,0.092772,-0.067860,0.0,False,False,1
20758,you,augmented3-you_23.avi,28,0.523126,1.017326,0.030727,0.508542,1.014890,0.021320,0.503275,...,0.0,0.092772,-0.067860,0.0,0.092772,-0.067860,0.0,False,False,1


In [22]:
pd.concat([df, df_augmented]).to_csv("datasets/libras_minds_dataset_openpose_augmented.csv")

In [126]:
df_augmented.to_csv("datasets/lsa64_augmentation_test.csv")